# Splitting trajectories

<img align="right" src="https://anitagraser.github.io/movingpandas/pics/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/movingpandas-examples/main?filepath=1-tutorials/6-splitting-trajectories.ipynb)

Gaps are quite common in trajectories. For example, GPS tracks may contain gaps if moving objects enter tunnels where GPS reception is lost. In other use cases, moving objects may leave the observation area for longer time before returning and continuing their recorded track.

Depending on the use case, we therefore might want to split trajectories at observation gaps that exceed a certain minimum duration:

In [ ]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import movingpandas as mpd

print(mpd.__version__)

In [ ]:
gdf = read_file('../data/geolife_small.gpkg')
gdf['t'] = pd.to_datetime(gdf['t'])
# Create fields to help vizualise splitting trajectories
gdf["minutes_since_trajectory_start"] = gdf.groupby("trajectory_id")["t"].transform(lambda x: (x - x.min()).dt.seconds / 60)
gdf["minutes_since_last_observation"] = gdf.groupby("trajectory_id")["minutes_since_trajectory_start"].diff()
gdf = gdf.set_index('t').tz_localize(None)
traj_collection = mpd.TrajectoryCollection(gdf, 'trajectory_id')

In [ ]:
my_traj = traj_collection.trajectories[1]
print(my_traj)
my_traj.plot(linewidth=5.0, capstyle='round', column='speed', vmax=20)

## ObservationGapSplitter

In [ ]:
help(mpd.ObservationGapSplitter)

Split the trajectory where then are no observations for at least 5 minutes

In [ ]:
split = mpd.ObservationGapSplitter(my_traj).split(gap=timedelta(minutes=5))
split

In [ ]:
for traj in split:
    print(traj)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=len(split), figsize=(19,4))
for i, traj in enumerate(split):
    traj.plot(ax=axes[i], linewidth=5.0, capstyle='round', column='minutes_since_trajectory_start', legend=True)

## StopSplitter

In [ ]:
help(mpd.StopSplitter)

In [ ]:
split = mpd.StopSplitter(my_traj).split(min_duration=timedelta(minutes=1), max_diameter=30, min_length=500)
split

In [ ]:
for traj in split:
    print(traj)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=len(split), figsize=(19,4))
for i, traj in enumerate(split):
    traj.plot(ax=axes[i], linewidth=5.0, capstyle='round', column='speed', vmax=20)